 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Od autorki: Dołączony jest plik konfigurtacyjny środowiska wirutalnego, aby można było wykonać całe zadanie w odpowiednio przygotowanym środowisku. Należy najpierw aktywować środowisko robocze.

In [ ]:
# Tutaj odtwórz i aktywuj środowisko robocze

 Tutaj zaimportuj wymagane biblioteki

In [8]:
import requests
import csv
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from time import sleep

 Tutaj zdefiniuj paramatry połączenia do API

In [9]:
# klucz
def get_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    
    return api_key


path_api = "../API_KEY.txt"  # ścieżka do pliku

API_KEY = get_api_key(path_api)  #pobranie klucza

In [10]:
# adres API bez endpointów
URL = 'https://api-datalab.coderslab.com/api/v2'

In [11]:
# funkcja składająca URL do endopointu Airport
def get_airport_URL(airport_id):
    return f'{URL}/airport/{airport_id}'

 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [12]:
airport_file_path = "../data/airports.csv"
airports_list = []
with open(airport_file_path, newline="") as my_file:
    a_ids = csv.reader(my_file)
    next(a_ids)
    for line in a_ids:
        for element in line:
            airports_list.append(element)


In [13]:
print(airports_list[:10])

['10874', '11233', '13360', '15008', '11638', '14150', '15323', '14814', '12007', '11337']


In [14]:
# sprawdzam, czy nie powtarzają sie elementy
if len(set(airports_list)) < len(airports_list):
    print("W liście są powtórzone elementy.")
else:
    print("W liście nie ma powtórzonych elementów.")

W liście nie ma powtórzonych elementów.


Notatka: Dobrą praktyką na przyszłość (z biznesowego punktu widzenia) byłoby tutaj dodanie mechanizmu sprawdzającego czy dane, które chcemy pobrać z APi są już w bazie (już pobrane) - żeby przy kolejnym wywołaniu kodu np. w następnym cyklu o innych datach krańcowych, nie pobierało tego, co już w bazie jest, ale sprawdzało, czy nie doszły nowe dane. Może być przecież tak, że dla jakiegoś id zaczęto zbierać dane w późniejszym czasie, więc nie będzie wszystkich, ale powinny być uwzględnione w końcowym raporcie. Zależy od wymagań klienta biznesowego.

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [15]:
# na próbę pobranie jednego elementu
response = requests.get(get_airport_URL(11638),headers={'authorization':API_KEY})
if response.ok == True:
    print(response.json())
else:
    print("Bad request")

{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}


In [16]:
def call_airport_API(airport_id):
    response = requests.get(get_airport_URL(airport_id),headers={'authorization':API_KEY})
    if response.ok == True:
        return (response.json())  # tu ma zwracać słownik, który będziemy zapisywać
    else:
        return None  # tu ma zwracać None

In [17]:
# # Teraz zrobimy pętlę, która będzie dodawać słowniki dla każdego lotniska do listy, jeśli danego id nie ma w endpointcie to będzie 'continue'. 
# Po zrobieniu listy całą ją zapiszemy do df za pomocą pandas.DataFrame.from_records

In [18]:
airport_responses = []  # lista słowników z danymi lotnisk
missing_airports = []  # na wszelki wypadek tworzę listę lotnisk, których brakuje w API
for item in airports_list:
    airport_dict = call_airport_API(item)
    if airport_dict is None:
        missing_airports.append(item)  # dodaje lotnisko do listy brakujących
    else:
        airport_responses.append(airport_dict)  # dodaje słownik dla lotniska do listy lotnisk

In [19]:
print(airport_responses[:5])

[{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}, {'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}, {'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}, {'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}, {'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}]


In [20]:
print("Brakujące: ", len(missing_airports))
print("Istniejące: ", len(airport_responses))
print("Wszystkie: ", len(airports_list))

Brakujące:  267
Istniejące:  97
Wszystkie:  364


In [21]:
# metoda .from_records jest ogłoszona jako przestarzała, więc nie będę jej używać, 
# zamiast tego polecane jest po prostu użycie pandas.DataFrame
airport_df = pd.DataFrame(airport_responses)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [22]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

In [23]:
airport_df.to_csv(r'..\data\raw\airport_list.csv', index=False)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [24]:
def call_airport_weather_API(date):
    response = requests.get(f'{URL}/airportWeather?date={date}',headers={'authorization':API_KEY})
    if response.ok == True:
        return response.json() # tu ma zwracać słownik, który będziemy zapisywać
    else:
        print(f"Bad request for date {date}")  # tu ma zwracać None
        return None

In [25]:
date_for_API = datetime.date(2019, 1, 1)
call_airport_weather_API(date_for_API)  # sprawdzam, czy zadziała format daty

[{'WT18': None,
  'STATION': 'USW00013874',
  'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US',
  'DATE': '2019-01-01',
  'AWND': 4.7,
  'PRCP': 0.14,
  'SNOW': 0,
  'SNWD': 0,
  'TAVG': 64,
  'TMAX': 66,
  'TMIN': 57,
  'WDF2': 310,
  'WDF5': 310,
  'WSF2': 15,
  'WSF5': 19,
  'WT01': 1},
 {'WT18': None,
  'STATION': 'USW00013874',
  'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US',
  'DATE': '2019-01-02',
  'AWND': 4.92,
  'PRCP': 0.57,
  'SNOW': 0,
  'SNWD': 0,
  'TAVG': 56,
  'TMAX': 59,
  'TMIN': 49,
  'WDF2': 70,
  'WDF5': 50,
  'WSF2': 12.1,
  'WSF5': 15,
  'WT01': 1,
  'WT08': 1},
 {'WT18': None,
  'STATION': 'USW00013874',
  'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US',
  'DATE': '2019-01-03',
  'AWND': 5.37,
  'PRCP': 0.15,
  'SNOW': 0,
  'SNWD': 0,
  'TAVG': 52,
  'TMAX': 55,
  'TMIN': 51,
  'WDF2': 340,
  'WDF5': 330,
  'WSF2': 15,
  'WSF5': 18.1,
  'WT01': 1},
 {'WT18': None,
  'STATION': 'USW00013874',
  'NAME': 'ATLA

In [26]:
weather_reports_list = []

for i in range(15):
    weather_report = call_airport_weather_API(date_for_API)
    for element in weather_report:
        weather_reports_list.append(element)
    print(datetime.datetime.now(), " Pobrane dane dla miesiąca: ", date_for_API, f" Obieg pętli: {i}")  #printujemy logi
    date_for_API += relativedelta(months=1)
    

2024-03-07 09:06:53.379315  Pobrane dane dla miesiąca:  2019-01-01  Obieg pętli: 0
2024-03-07 09:06:54.197918  Pobrane dane dla miesiąca:  2019-02-01  Obieg pętli: 1
2024-03-07 09:06:55.032642  Pobrane dane dla miesiąca:  2019-03-01  Obieg pętli: 2
2024-03-07 09:06:55.828225  Pobrane dane dla miesiąca:  2019-04-01  Obieg pętli: 3
2024-03-07 09:06:56.616652  Pobrane dane dla miesiąca:  2019-05-01  Obieg pętli: 4
2024-03-07 09:06:57.402235  Pobrane dane dla miesiąca:  2019-06-01  Obieg pętli: 5
2024-03-07 09:06:58.310699  Pobrane dane dla miesiąca:  2019-07-01  Obieg pętli: 6
2024-03-07 09:06:59.116435  Pobrane dane dla miesiąca:  2019-08-01  Obieg pętli: 7
2024-03-07 09:06:59.905793  Pobrane dane dla miesiąca:  2019-09-01  Obieg pętli: 8
2024-03-07 09:07:00.724541  Pobrane dane dla miesiąca:  2019-10-01  Obieg pętli: 9
2024-03-07 09:07:01.601470  Pobrane dane dla miesiąca:  2019-11-01  Obieg pętli: 10
2024-03-07 09:07:02.476474  Pobrane dane dla miesiąca:  2019-12-01  Obieg pętli: 11
20

In [27]:
print(len(weather_reports_list))

46226


In [28]:
airport_weather_df = pd.DataFrame(weather_reports_list)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [29]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

In [30]:
airport_weather_df.to_csv(r'..\data\raw\airport_weather.csv', index=False)

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [31]:
def call_airport_aircraft_API():
    response = requests.get(f'{URL}/aircraft',headers={'authorization':API_KEY})
    if response.ok == True:
        return response.json() # tu ma zwracać słownik, który będziemy zapisywać
    else:
        print("Bad request")  # tu ma zwracać None
        return None

In [32]:
planes_list = []
aircraft_info = call_airport_aircraft_API()
for item in aircraft_info:
    planes_list.append(item)

In [33]:
print(len(planes_list))

7383


In [34]:
aircraft_df = pd.DataFrame(planes_list)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [35]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

In [36]:
aircraft_df.to_csv(r'..\data\raw\aircraft.csv', index=False)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

In [37]:
# data początkowa:
date_starting = datetime.date(2019, 1, 1)

def change_date_format(input_date):  # funkcja zmieniająca format daty
    return input_date.strftime('%Y-%m')

In [41]:
# próba dla wewnętrznej pętli (czyli dla jednego lotniska dane z 15 miesięcy)
temp_id = '14814'
def call_flights_by_airportid(temp_id, date_to_chceck):  #wywołanie API dla jednego lotniska i konkretnej daty
    date_formatted = change_date_format(date_to_chceck)
    response = requests.get(f'{URL}/flight?airportId={temp_id}&date={date_formatted}',headers={'authorization':API_KEY})
    if response.status_code == 200:
        print(datetime.datetime.now(), f" Pobrano dane dla lotniska o id: {temp_id} w miesiącu {date_to_chceck}")
        return response.json()
    elif response.status_code == 204:
        print(datetime.datetime.now(), f" Brak danych dla lotniska o id: {temp_id} w miesiącu {date_to_chceck}")
        return None
    else:
        print("Bad request: ", response.status_code)  # tu ma zwracać None
        return None
            
def one_airport_loop(temp_id):  # pętla dla jednego lotniska, wykorzystująca funkcję zdefiniowaną powyżej
    temp_list = []
    date = date_starting
    for i in range(15):
        flights = call_flights_by_airportid(temp_id, date)
        date += relativedelta(months=1)  # zwiększam datę o miesiąc do następnego obiegu pętli
        if flights is None:
            continue
        else:
            temp_list.extend(flights)
    print(datetime.datetime.now(), f"Koniec zadania dla lotniska {temp_id}")
    if len(temp_list) == 0:
        return None
    else:
        return temp_list

In [42]:
jedno_lotnisko = one_airport_loop(11278)  # sprawdzam działanie swoich funkcji
print(jedno_lotnisko[:5])

2024-03-07 09:19:48.845789  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-01-01
2024-03-07 09:19:50.790232  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-02-01
2024-03-07 09:19:52.864184  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-03-01
2024-03-07 09:19:55.018750  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-04-01
2024-03-07 09:19:57.231784  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-05-01
2024-03-07 09:19:59.421185  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-06-01
2024-03-07 09:20:01.648831  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-07-01
2024-03-07 09:20:03.701855  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-08-01
2024-03-07 09:20:05.877651  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-09-01
2024-03-07 09:20:08.063483  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-10-01
2024-03-07 09:20:10.178660  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-11-01
2024-03-07 09:20:12.292497  Pobr

In [43]:
# pętla dla 2 lotnisk - sprawdzimy działanie ze sleep
# 12264	Washington Dulles International
# 11278	Ronald Reagan Washington National

airports_to_test = ['11278', '12264']
test_list = []
for var in airports_to_test:
    test_response = one_airport_loop(var)
    sleep(5)
    if test_response != None:
        test_list.extend(test_response)
    print(datetime.datetime.now(), "Przechodzę do następnego lotniska")

2024-03-07 09:20:29.046165  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-01-01
2024-03-07 09:20:31.147019  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-02-01
2024-03-07 09:20:33.287531  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-03-01
2024-03-07 09:20:35.422066  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-04-01
2024-03-07 09:20:37.567180  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-05-01
2024-03-07 09:20:39.667473  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-06-01
2024-03-07 09:20:41.868439  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-07-01
2024-03-07 09:20:43.902792  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-08-01
2024-03-07 09:20:46.104651  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-09-01
2024-03-07 09:20:48.283157  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-10-01
2024-03-07 09:20:50.338705  Pobrano dane dla lotniska o id: 11278 w miesiącu 2019-11-01
2024-03-07 09:20:52.355764  Pobr

In [45]:
len(test_list)  # wynik z pustą listą 38354

38354

In [46]:
output_list = []
for airport_id in airports_list:
    test_response = one_airport_loop(airport_id)
    sleep(5)
    if test_response != None:
        output_list.extend(test_response)
    print(datetime.datetime.now(), "Przechodzę do następnego lotniska")
 
# with open(r'..\data\raw\temp_flights.csv', "w", newline="") as plik:  #zabezpieczenie ;)
#   writer = csv.writer(plik)
#   writer.writerows(output_list)

2024-03-07 09:23:42.050096  Brak danych dla lotniska o id: 10874 w miesiącu 2019-01-01
2024-03-07 09:23:43.377732  Brak danych dla lotniska o id: 10874 w miesiącu 2019-02-01
2024-03-07 09:23:44.691701  Brak danych dla lotniska o id: 10874 w miesiącu 2019-03-01
2024-03-07 09:23:45.982767  Brak danych dla lotniska o id: 10874 w miesiącu 2019-04-01
2024-03-07 09:23:47.375014  Brak danych dla lotniska o id: 10874 w miesiącu 2019-05-01
2024-03-07 09:23:48.584128  Brak danych dla lotniska o id: 10874 w miesiącu 2019-06-01
2024-03-07 09:23:49.860903  Brak danych dla lotniska o id: 10874 w miesiącu 2019-07-01
2024-03-07 09:23:51.127535  Brak danych dla lotniska o id: 10874 w miesiącu 2019-08-01
2024-03-07 09:23:52.643370  Brak danych dla lotniska o id: 10874 w miesiącu 2019-09-01
2024-03-07 09:23:53.972391  Brak danych dla lotniska o id: 10874 w miesiącu 2019-10-01
2024-03-07 09:23:55.279142  Brak danych dla lotniska o id: 10874 w miesiącu 2019-11-01
2024-03-07 09:23:56.620343  Brak danych dla

In [47]:
len(output_list)

1386120

In [48]:
flight_df = pd.DataFrame(output_list)

In [ ]:
# ##Zapis poprawnych danych, bo wczesniej ccos ci nie wyszlo
# keys = output_list[0].keys()
# with open(r'..\data\raw\temp_flights_by_piotr.csv', 'w', newline='') as output_file:
#     dict_writer = csv.DictWriter(output_file, fieldnames=keys,extrasaction='ignore', delimiter = ';')
#     dict_writer.writeheader()
#     dict_writer.writerows(output_list)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [49]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`

In [50]:
flight_df.to_csv(r'..\data\raw\flight.csv', index=False)

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [51]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.
